<a href="https://colab.research.google.com/github/macij1/Machine-Learning-Project/blob/main/src/ML_120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, KFold

from sklearn import svm, tree, neighbors, neural_network, ensemble, preprocessing

from sklearn.metrics import confusion_matrix, fowlkes_mallows_score
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import mutual_info_score, adjusted_rand_score

from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict

from sklearn.cluster import KMeans

In [6]:
#Get data and show that it has done so, Note you will have to upload the data if opened on google Colab, file in teams chat

data = pd.read_excel('Grade_Distribution_Data.xlsx', sheet_name='AY2023 AY2024 Grade Distro')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Grade_Distribution_Data.xlsx'

In [4]:
#Binary encoding

def binary_encode_column(df, column_name):
    # Get unique values in the column and the number of bits required
    unique_values = df[column_name].unique()
    num_bits = int(np.ceil(np.log2(len(unique_values))))  # Minimum bits needed

    # Map each unique value to its binary representation
    binary_mapping = {val: format(i, f'0{num_bits}b') for i, val in enumerate(unique_values)}

    # Apply the mapping to create a new column with binary strings
    df[f"{column_name}_bin"] = df[column_name].map(binary_mapping)

    # Drop the original column if desired
    df = df.drop(columns=[column_name])

    return binary_mapping

In [3]:
#Begin preprocessing here
#There is a lot more that can be done with pandas for manipulating our data, this is just a start

train_data = data.copy(deep="True")

np.random.seed(100)

train_data.drop(columns=['A', 'B', 'C', 'D', 'F', 'W', 'Section', 'Trm Code'], inplace=True)
train_data.rename(columns={'Academic Year': 'AcademicYear', 'Course Subject and Number': 'CourseSubjectandNumber', 'Average Grade': 'AverageGrade', 'Primary Instructor Name': 'PrimaryInstructorName'}, inplace=True)
train_data = train_data[train_data.AcademicYear != "2022-23"]
train_data = train_data[train_data.AverageGrade != 4]
train_data = train_data[train_data.AverageGrade != 'Total']
train_data.drop(columns=['AcademicYear'], inplace=True)
#train_data = train_data[train_data['CourseSubjectandNumber'].str.contains("CS")] <-- we have no CS classes lol
train_data.dropna(inplace=True)

#Begin converting all strings into ints, this require a lot of encoding and will be difficult for instructors
train_data[['Subject', 'Number']] = train_data['CourseSubjectandNumber'].str.split(' ', n=1, expand=True)
train_data.drop(columns=['CourseSubjectandNumber'], inplace=True)
train_data.rename(columns={'PrimaryInstructorName': 'Instructor'}, inplace=True)
print(train_data.columns)

columns_map = {'Subject': {'AE': 0, 'ARCH': 1, 'ECE': 2, 'ME': 3, 'NRE': 4, 'AE': 5, 'MP': 6}}
train_data.replace(columns_map, inplace=True)

#Temporary insert of answers for now, plus randomizer, should do this manually or something later, 0 = easy, 1 = hard
train_data.insert(len(train_data.columns), "Difficulty", 0)
change = train_data.sample(int(0.2*len(train_data))).index
train_data.loc[change,'Difficulty'] = 1

# TODO: Recover name with the mapping
instructor_encode_map = binary_encode_column(train_data, 'Instructor')
train_data.drop(columns=['Instructor'], inplace=True)


#Easy way to split into train and test, only train is used right now
train, test = np.split(train_data.sample(frac=1), [int(0.8*len(train_data))])

X_train = train.loc[:, train_data.columns != 'Difficulty']
y_train = train.loc[:, 'Difficulty']

NameError: name 'data' is not defined

In [ ]:
X_train

,AverageGrade,Standard Deviation,Subject,Number,Instructor_bin
2476,2,0.894427,2,4902,010001010
3434,3.23,0.749613,2,7750,001101111
2046,3.84,1.390153,5,3531,000001111
3125,3.75,1.299038,1,4016,000110111
3568,3.75,1.310276,3,4042,011011101
...,...,...,...,...,...
2951,3.69,1.363470,5,4040,000100110
3656,2.88,0.624695,3,4790,000110001
3878,3.97,1.727661,1,4128,101101110
3543,3.41,0.976398,3,3210,011001110


In [ ]:
y_train

,Difficulty
2476,0
3434,0
2046,0
3125,0
3568,1
...,...
2951,0
3656,0
3878,0
3543,0


In [ ]:
#KMeans

model = KMeans(n_clusters=2, random_state=0, n_init="auto") #Basic, no parameter tweaking really

km_model = model.fit(X_train.values, y_train.values) #Need everything to be numbers instead of strings

# confusion matrix
KM_y_pred = cross_val_predict(model, X_train, y_train) #Default parameters once again, perhaps specify number of folds?

KM_tn, KM_fp, KM_fn, KM_tp = confusion_matrix(y_train, KM_y_pred).ravel()
print("True Negatives", KM_tn)
print("False Positives", KM_fp)
print("False Negatives", KM_fn)
print("True Positives", KM_tp)

silhouette = silhouette_score(X_train, km_model.labels_)
db_index = davies_bouldin_score(X_train, km_model.labels_)
ch_index = calinski_harabasz_score(X_train, km_model.labels_)
print(f"Silhouette Score: {silhouette:.2f}")
print(f"Davies-Bouldin Index: {db_index:.2f}")
print(f"Calinski-Harabasz Index: {ch_index:.2f}")

fm_score = fowlkes_mallows_score(y_train, km_model.labels_)
print(f"Fowlkes-Mallows Score: {fm_score:.2f}")



True Negatives 573
False Positives 129
False Negatives 152
True Positives 21
Silhouette Score: 0.95
Davies-Bouldin Index: 0.06
Calinski-Harabasz Index: 53939.93
Fowlkes-Mallows Score: 0.69
